In [1]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
import pandas as pd

****Heterologous expression of 5-HTP and MH4 pathway in Bacillus subtilis****

In [2]:
model = read_sbml_model('iYO844.xml')

In [3]:
model

Name,iYO844
Memory address,0x07ff3e0035be0
Number of metabolites,990
Number of reactions,1250
Number of groups,0
Objective expression,1.0*BIOMASS_BS_10 - 1.0*BIOMASS_BS_10_reverse_8788b
Compartments,"cytosol, extracellular space"


In [4]:
for metabolite in model.metabolites.query('Formate', 'name'):
    print(metabolite.name, metabolite.id)

Formate for_c
Formate for_e


In [5]:
for metabolite in model.metabolites.query('GTP', 'name'):
    print(metabolite.name, metabolite.id)

DGTP C10H12N5O13P3 dgtp_c
GTP C10H12N5O14P3 gtp_c


In [6]:
for metabolite in model.metabolites.query('H2O', 'name'):
    print(metabolite.name, metabolite.id)

H2O H2O h2o_c
H2O H2O h2o_e
Oxaloglycolate C4H2O6 oxglyc_c
Sulfoacetate C2H2O5S sula_c
Sulfoacetate C2H2O5S sula_e


In [9]:
for metabolite in model.metabolites.query('Inorganic triphosphate', 'name'):
    print(metabolite.name, metabolite.id, metabolite.formula)

Inorganic triphosphate pppi_c HO10P3


In [16]:
for metabolite in model.metabolites.query('Nicotinamide', 'name'):
    print(metabolite.name, metabolite.id)

Nicotinamide adenine dinucleotide nad_c
Nicotinamide adenine dinucleotide - reduced nadh_c
Nicotinamide adenine dinucleotide phosphate nadp_c
Nicotinamide adenine dinucleotide phosphate - reduced nadph_c
Nicotinamide ncam_c


In [17]:
model.metabolites.h_c 

Metabolite identifier,h_c
Name,H+
Memory address,0x07ff3c14787c0
Formula,H
Compartment,c
In 616 reaction(s),"PNTOt2, SADT, KAS6, LAAL6, D4DGCD, ADPRDP, CITt11, LAAL17, DPCOAK, LIPO3S24_BS, FORt2, GLGC, BUTSabc, AGPR, CYTOM, KAS8, CITt12, DAGK_BS, CBPS, ETOHt3, GLYCK, P5CD, CITt10, GLCNt2ir, 3CMPt6, KAS9,..."


In [19]:
for metabolite in model.metabolites.query('pterin', 'name'):
    print(metabolite.name, metabolite.id, metabolite.formula)

6-Pyruvoyl-5,6,7,8-tetrahydropterin 6pthp_c C9H11N5O3
Dihydroneopterin dhnpt_c C9H13N5O4


In [20]:
for metabolite in model.metabolites.query('Tryptophan', 'name'):
    print(metabolite.name, metabolite.id)

L-Tryptophan trp__L_c
L-Tryptophan trp__L_e


In [23]:
for metabolite in model.metabolites.query('O2 O2', 'name'):
    print(metabolite.name, metabolite.id)

O2 O2 o2_c
O2 O2 o2_e


In [24]:
MH2_3P = Metabolite(id='MH2_3P', compartment='c')

In [25]:
preMH2_3P = Metabolite(id='preMH2_3P', compartment='c')

In [26]:
MH2 = Metabolite(id='MH2', compartment='c')

In [27]:
MH4 = Metabolite(id='MH4', compartment='c')

In [28]:
HydroxyMH4 = Metabolite(id='hydroxy_MH4', compartment='c')

In [29]:
hydroxy_trp = Metabolite (id='hydroxy_trp', compartment='c')

In [31]:
folE = Reaction('R05048')
folE.add_metabolites({model.metabolites.h2o_c: -1,
                      model.metabolites.gtp_c: -1,
                      model.metabolites.for_c: 1,
                      preMH2_3P: 1
                     })
print(folE.build_reaction_string())

gtp_c + h2o_c --> for_c + preMH2_3P


In [32]:
folX = Reaction('R11082')
folX.add_metabolites({preMH2_3P: -1,
                      MH2_3P: 1
                     })
print(folX.build_reaction_string())

preMH2_3P --> MH2_3P


In [33]:
MH2_3P_dephosphorylation = Reaction ('Putative_dephosphorylation')
MH2_3P_dephosphorylation.add_metabolites({MH2_3P: -1,
                                          MH2: 1,
                                          model.metabolites.pppi_c: 1
                                         })
print(MH2_3P_dephosphorylation.build_reaction_string())

MH2_3P --> MH2 + pppi_c


In [34]:
folM = Reaction('R11019')
folM.add_metabolites ({model.metabolites.nadph_c: -1,
                       model.metabolites.h_c: -1,
                       MH2: -1,
                       MH4: 1,
                       model.metabolites.nadp_c: 1
                      })
print(folM.build_reaction_string())

MH2 + h_c + nadph_c --> MH4 + nadp_c


In [35]:
phhB = Reaction ('R04734')
phhB.add_metabolites({HydroxyMH4: -1,
                      MH2: 1,
                      model.metabolites.h2o_c: 1
                     })
print(phhB.build_reaction_string())

hydroxy_MH4 --> MH2 + h2o_c


In [36]:
phhA = Reaction ('R12540')
phhA.add_metabolites({model.metabolites.trp__L_c: -1,
                      model.metabolites.o2_c: -1,
                      MH4: -1,
                      HydroxyMH4: 1,
                      hydroxy_trp: 1
                     })
print(phhA.build_reaction_string())

MH4 + o2_c + trp__L_c --> hydroxy_MH4 + hydroxy_trp


In [37]:
model.add_reactions([folE])

In [38]:
model.add_reactions([folX])

In [39]:
model.add_reactions([MH2_3P_dephosphorylation])

In [40]:
model.add_reactions([folM])

In [41]:
model.add_reactions([phhB])

In [42]:
model.add_reactions([phhA])

In [43]:
model.reactions.R12540

Reaction identifier,R12540
Name,
Memory address,0x07ff3c0b06700
Stoichiometry,MH4 + o2_c + trp__L_c --> hydroxy_MH4 + hydroxy_trp + O2 O2 + L-Tryptophan --> +
GPR,
Lower bound,0.0
Upper bound,1000.0


In [44]:
hydroxy_5HTP_exchange = Reaction('EX_hydroxy_trp')
hydroxy_5HTP_exchange.add_metabolites({model.metabolites.hydroxy_trp: -1})
model.add_reactions([hydroxy_5HTP_exchange])

In [45]:
model.optimize()

,fluxes,reduced_costs
EX_etha_e,0.0,-0.066773
EX_drib_e,0.0,-0.150240
EX_csn_e,0.0,-0.129837
EX_5mtr_e,0.0,-0.224433
EX_dtmp_e,0.0,-0.337577
...,...,...
Putative_dephosphorylation,0.0,0.000000
R11019,0.0,0.000000
R04734,0.0,-0.012984
R12540,0.0,-0.391366


In [46]:
with model:
    print(model.reactions.R12540)
    model.objective = model.reactions.R12540
    max_5HTP_production = model.optimize().objective_value
    print(max_5HTP_production)

R12540: MH4 + o2_c + trp__L_c --> hydroxy_MH4 + hydroxy_trp
0.5776145203111468
